In [1]:
import sys, os # for running from repo
sys.path.insert(0, os.path.abspath("../../"))

try: from brmspy import brms
except ImportError:
    %pip install -q brmspy
    from brmspy import brms

[brmspy][_call_with_frames_removed] Running autoload!
[brmspy][_autoload] Activating runtime /Users/sebastian/.brmspy/runtime/macos-arm64-r4.5-0.2.0
[brmspy][_autoload] lib paths are ['/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/library', '/Users/sebastian/.brmspy/runtime/macos-arm64-r4.5-0.2.0/Rlib']
[brmspy][_autoload] Setting cmdstan path to /Users/sebastian/.brmspy/runtime/macos-arm64-r4.5-0.2.0/cmdstan


In [2]:
#with brms.manage() as ctx:
#    ctx.install_brms(use_prebuilt=True)

In [3]:
from brmspy.brms import bf, poisson, set_rescor

In [4]:
epilepsy = brms.get_brms_data("epilepsy")
epilepsy.index = [str(f"obs{i}") for i in range(len((epilepsy)))]
formula = "count ~ zAge + zBase * Trt + (1|patient)"
family = "poisson"

formula_multi = (bf(
  "count ~ zAge + zBase * Trt + visit + (1 + visit | patient)"
) + poisson()) + \
(bf(
  "Base ~ zAge + Trt + (1 | patient)"
) + poisson()) + \
set_rescor(False)

/Users/sebastian/PycharmProjects/pybrms/brmspy/_session/codec/builtin.py:259: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.blocks_columns = columns_metadata


In [5]:
epilepsy['Trt'] = epilepsy['Trt'].astype("category")

In [6]:
print(epilepsy.head().to_string())

       Age  Base Trt  patient  visit  count  obs      zAge     zBase
obs0  31.0  11.0   1        1      1    5.0    1  0.424995 -0.757173
obs1  30.0  11.0   1        2      1    3.0    2  0.265284 -0.757173
obs2  25.0   6.0   1        3      1    2.0    3 -0.533274 -0.944403
obs3  36.0   8.0   1        4      1    4.0    4  1.223553 -0.869511
obs4  22.0  66.0   1        5      1    7.0    5 -1.012408  1.302363


In [7]:
m_uni = brms.brm(
    formula=formula,
    data=epilepsy,
    family=family,
    warmup=100,
    iter=200,
    chains=4
)
id_uni = m_uni.idata

/Users/sebastian/PycharmProjects/pybrms/brmspy/_session/codec/builtin.py:259: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.blocks_columns = columns_metadata
[brmspy][worker_main] Fitting model with brms (backend: cmdstanr)...
Running MCMC with 4 chains, at most 2 in parallel...

Chain 1 WARNING: There aren't enough warmup iterations to fit the 
Chain 1          three stages of adaptation as currently configured. 
Chain 1          Reducing each adaptation stage to 15%/75%/10% of 
Chain 1          the given number of warmup iterations: 
Chain 1            init_buffer = 15 
Chain 1            adapt_window = 75 
Chain 1            term_buffer = 10 
Chain 1 Iteration:   1 / 200 [  0%]  (Warmup) 
Chain 2 WARNING: There aren't enough warmup iterations to fit the 
Chain 2          three stages of adaptation as currently configured. 
Chain 2          Reducing each adaptatio

In [8]:
m_multi = brms.brm(
    formula=formula_multi,
    data=epilepsy,
    warmup=100,
    iter=200,
    chains=4
)
id_multi = m_multi.idata

/Users/sebastian/PycharmProjects/pybrms/brmspy/_session/codec/builtin.py:259: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.blocks_columns = columns_metadata
[brmspy][worker_main] Fitting model with brms (backend: cmdstanr)...
Running MCMC with 4 chains, at most 2 in parallel...

Chain 1 WARNING: There aren't enough warmup iterations to fit the 
Chain 1          three stages of adaptation as currently configured. 
Chain 1          Reducing each adaptation stage to 15%/75%/10% of 
Chain 1          the given number of warmup iterations: 
Chain 1            init_buffer = 15 
Chain 1            adapt_window = 75 
Chain 1            term_buffer = 10 
Chain 1 Iteration:   1 / 200 [  0%]  (Warmup) 
Chain 2 WARNING: There aren't enough warmup iterations to fit the 
Chain 2          three stages of adaptation as currently configured. 
Chain 2          Reducing each adaptatio

In [9]:
import xarray as xr

m = m_uni

idata_post = brms.posterior(m).idata         # groups: posterior + constant_data
idata_obs  = brms.observed_data(m).idata     # groups: observed_data + constant_data
idata_ll   = brms.log_lik(m).idata           # groups: log_likelihood + constant_data

# optional: in-sample mean + linpred (stored in posterior)
idata_mu   = brms.posterior_epred(m).idata   # groups: posterior + constant_data   (var: count_mean)
idata_eta  = brms.posterior_linpred(m).idata # groups: posterior + constant_data   (var: count_linpred)

# 3) Compose groups that don't overlap using extend()
idata = idata_post
idata.extend(idata_obs)   # adds observed_data
idata.extend(idata_ll)    # adds log_likelihood

# 4) Compose within-group posterior variables using xarray merge
# (because extend() won't merge inside an existing group)
idata.posterior = xr.merge([idata.posterior, idata_mu.posterior, idata_eta.posterior])

print(idata.groups())
# -> ['posterior', 'observed_data', 'log_likelihood', 'constant_data']
print(idata)

['posterior', 'log_likelihood', 'observed_data', 'constant_data']
Inference data with groups:
	> posterior
	> log_likelihood
	> observed_data
	> constant_data


In [10]:
idata

Inference data with groups:
	> posterior
	> log_likelihood
	> observed_data
	> constant_data

In [11]:
sample = epilepsy.sample(322, replace=True).reset_index(drop=True).copy()

In [12]:
D = {}
for model, model_type in zip([m_uni, m_multi], ['uni', 'multi']):
    p = brms.posterior(model).idata
    ep = brms.posterior_epred(model).idata
    ep_new = brms.posterior_epred(model, sample).idata

    pp = brms.posterior_predict(model).idata
    pp_new = brms.posterior_predict(model, newdata=sample).idata
    lp = brms.posterior_linpred(model).idata
    lp_new = brms.posterior_linpred(model, sample).idata
    ll = brms.log_lik(model).idata
    ll_new = brms.log_lik(model, sample).idata

    D.update({
        f"{model_type}: posterior(m)": p,
        f"{model_type}: posterior_epred(m)": ep,
        f"{model_type}: posterior_epred(m, newdata)": ep_new,
        f"{model_type}: posterior_predict(m)": pp,
        f"{model_type}: posterior_predict(m, newdata)": pp_new,
        f"{model_type}: posterior_linpred(m)": lp,
        f"{model_type}: posterior_linpred(m, newdata)": lp_new,
        f"{model_type}: log_lik(m)": ll,
        f"{model_type}: log_lik(m, newdata)": ll_new
    })

/Users/sebastian/PycharmProjects/pybrms/brmspy/_session/codec/builtin.py:259: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.blocks_columns = columns_metadata
/Users/sebastian/PycharmProjects/pybrms/brmspy/_session/codec/builtin.py:259: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.blocks_columns = columns_metadata
/Users/sebastian/PycharmProjects/pybrms/brmspy/_session/codec/builtin.py:259: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.blocks_columns = columns_metadata
/Users/sebastian/PycharmProjects/pybrms/brmspy/_session/codec/builtin.py:259: UserWarning: Pandas doesn't allow columns to be created via a new attribute name -

In [13]:
epilepsy.index

Index(['obs0', 'obs1', 'obs2', 'obs3', 'obs4', 'obs5', 'obs6', 'obs7', 'obs8',
       'obs9',
       ...
       'obs226', 'obs227', 'obs228', 'obs229', 'obs230', 'obs231', 'obs232',
       'obs233', 'obs234', 'obs235'],
      dtype='object', length=236)

In [14]:
for model, model_type in zip([m_uni, m_multi], ['uni', 'multi']):
    print("model_type:", model_type)
    print("idata:")
    for key in list(model.idata.keys()):
        print(key)
        print(model.idata[key])

        print("\n"*1)

    print("\n"*3)

model_type: uni
idata:
posterior
<xarray.Dataset> Size: 218kB
Dimensions:                  (chain: 4, draw: 100)
Coordinates:
  * chain                    (chain) int64 32B 0 1 2 3
  * draw                     (draw) int64 800B 0 1 2 3 4 5 ... 94 95 96 97 98 99
Data variables: (12/68)
    b_Intercept              (chain, draw) float64 3kB 2.135 2.021 ... 1.891
    b_zAge                   (chain, draw) float64 3kB 0.1688 0.1603 ... 0.06882
    b_zBase                  (chain, draw) float64 3kB 0.4272 0.7181 ... 0.4639
    b_Trt                    (chain, draw) float64 3kB -0.3606 ... -0.1797
    b_zBase:Trt              (chain, draw) float64 3kB 0.2189 ... 0.1963
    sd_patient__Intercept    (chain, draw) float64 3kB 0.5329 0.4626 ... 0.5605
    ...                       ...
    r_patient[56,Intercept]  (chain, draw) float64 3kB 1.295 1.133 ... 1.265
    r_patient[57,Intercept]  (chain, draw) float64 3kB -0.08514 ... -0.3228
    r_patient[58,Intercept]  (chain, draw) float64 3kB -1.59 

In [15]:
print("newdata shape:", sample.shape)
print("formula uni  :", formula)
print("formula multi:", formula_multi)

print("prediction outputs")
print("="*30)
for key, val in D.items():
    
    print(key)
    for k in val.groups():
        print(f"groups[{k}]")
        print(val[k])
        print("\n"*1)

newdata shape: (322, 9)
formula uni  : count ~ zAge + zBase * Trt + (1|patient)
formula multi: (bf('count ~ zAge + zBase * Trt + visit + (1 + visit | patient)') + poisson(link='log')) + (bf('Base ~ zAge + Trt + (1 | patient)') + poisson(link='log')) + set_rescor(rescor=False)
prediction outputs
uni: posterior(m)
groups[posterior]
<xarray.Dataset> Size: 218kB
Dimensions:                  (chain: 4, draw: 100)
Coordinates:
  * chain                    (chain) int64 32B 0 1 2 3
  * draw                     (draw) int64 800B 0 1 2 3 4 5 ... 94 95 96 97 98 99
Data variables: (12/68)
    b_Intercept              (chain, draw) float64 3kB 2.135 2.021 ... 1.891
    b_zAge                   (chain, draw) float64 3kB 0.1688 0.1603 ... 0.06882
    b_zBase                  (chain, draw) float64 3kB 0.4272 0.7181 ... 0.4639
    b_Trt                    (chain, draw) float64 3kB -0.3606 ... -0.1797
    b_zBase:Trt              (chain, draw) float64 3kB 0.2189 ... 0.1963
    sd_patient__Intercept    